from https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/convolutional_network_raw.py

In [1]:
import tensorflow as tf

In [ ]:
model_name = "cnn_tensorflow_10epochs.ckpt"
model_exists = False

In [2]:
learning_rate = 0.001
num_epochs = 10
batch_size = 10

In [3]:
input_height = 224
input_width = 224
channels = 3
num_classes = 6
dropout = 0.5

In [4]:
# Placeholders
X = tf.placeholder(tf.float32, [None, input_height, input_width, channels])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

In [5]:
# Variables
# weights: (kernel height, kernel width, input size, output size)
weights = {
    'wconv1': tf.Variable(tf.random_normal([3, 3, 3, 32])),
    'wconv2': tf.Variable(tf.random_normal([3, 3, 32, 32])),
    'wconv3': tf.Variable(tf.random_normal([3, 3, 32, 32])),
    'wconv4': tf.Variable(tf.random_normal([3, 3, 32, 32])),
    'wdense1': tf.Variable(tf.random_normal([56*56*32, 256])),
    'wout': tf.Variable(tf.random_normal([256, num_classes]))
}

# biases (output size)
biases = {
    'bconv1': tf.Variable(tf.random_normal([32])),
    'bconv2': tf.Variable(tf.random_normal([32])),
    'bconv3': tf.Variable(tf.random_normal([32])),
    'bconv4': tf.Variable(tf.random_normal([32])),
    'bdense1': tf.Variable(tf.random_normal([256])),
    'bout': tf.Variable(tf.random_normal([num_classes]))
}


In [6]:
# Create model
def convnet(x, weights, biases, dropout):
    
    # shape=(?, 224, 224, 32)
    conv1 = tf.nn.conv2d(x, weights['wconv1'], strides=[1, 1, 1, 1], padding='SAME')
    conv1 = tf.nn.bias_add(conv1, biases['bconv1'])
    conv1 = tf.nn.relu(conv1)
    
    # shape=(?, 224, 224, 32)
    conv2 = tf.nn.conv2d(conv1, weights['wconv2'], strides=[1, 1, 1, 1], padding='SAME')
    conv2 = tf.nn.bias_add(conv2, biases['bconv2'])
    conv2 = tf.nn.relu(conv2)
    # shape=(?, 112, 112, 32)
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    # shape=(?, 112, 112, 32)
    conv3 = tf.nn.conv2d(conv2, weights['wconv3'], strides=[1, 1, 1, 1], padding='SAME')
    conv3 = tf.nn.bias_add(conv3, biases['bconv3'])
    conv3 = tf.nn.relu(conv3)
    
    # shape=(?, 112, 112, 32)
    conv4 = tf.nn.conv2d(conv3, weights['wconv4'], strides=[1, 1, 1, 1], padding='SAME')
    conv4 = tf.nn.bias_add(conv4, biases['bconv4'])
    conv4 = tf.nn.relu(conv4)
    # shape=(?, 56, 56, 32)
    conv4 = tf.nn.max_pool(conv4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # Reshape conv4 output to fit fully connected layer input
    fc1 = tf.reshape(conv4, [-1, weights['wdense1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wdense1']), biases['bdense1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['wout']), biases['bout'])
    return tf.nn.softmax(out)


In [7]:
# Construct model
logits = convnet(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)


In [8]:
# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

In [9]:
# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


In [10]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'data/train'
test_data_dir = 'data/test' 

target_size = (224, 224)

train_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical')

#nxt = train_generator.next()
#nxt[0].shape, nxt[1].shape


Using TensorFlow backend.


Found 7096 images belonging to 6 classes.


In [17]:
saver = tf.train.Saver()

In [11]:
init = tf.global_variables_initializer()

n_train = 7096
steps_per_epoch = n_train // batch_size

In [12]:
with tf.Session() as sess:

    sess.run(init)

    for epoch in range(num_epochs):
        
        print("Starting epoch: {}".format(epoch))
        
        for step in range(steps_per_epoch):
            
            batch_x, batch_y = train_generator.next()
            sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: dropout})
            
        loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x, 
                                                             Y: batch_y,
                                                             keep_prob: 1.0})
        print("Epoch " + str(epoch) + ", Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")
    saver.save(sess, model_name)

   

Starting epoch: 0
Epoch 0, Loss= 1.8436, Training Accuracy= 0.200
Starting epoch: 1
Epoch 1, Loss= 1.8436, Training Accuracy= 0.200
Starting epoch: 2
Epoch 2, Loss= 1.8436, Training Accuracy= 0.200
Starting epoch: 3
Epoch 3, Loss= 2.0436, Training Accuracy= 0.000
Starting epoch: 4
Epoch 4, Loss= 1.8436, Training Accuracy= 0.200
Starting epoch: 5
Epoch 5, Loss= 2.0436, Training Accuracy= 0.000
Starting epoch: 6
Epoch 6, Loss= 1.8436, Training Accuracy= 0.200
Starting epoch: 7
Epoch 7, Loss= 1.7436, Training Accuracy= 0.300
Starting epoch: 8
Epoch 8, Loss= 1.6436, Training Accuracy= 0.400
Starting epoch: 9
Epoch 9, Loss= 1.7436, Training Accuracy= 0.300
Optimization Finished!


In [19]:
with tf.Session() as sess:
    sess.run(accuracy, 
         feed_dict={X: mnist.test.images[:256], Y: mnist.test.labels[:256],
         keep_prob: 1.0})


NameError: name 'mnist' is not defined